In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pandas as pd
import multiprocessing as mp
import sys
import threading
import re
import numpy as np

In [9]:
r = requests.get('http://www.britishpoliticalspeech.org/speech-archive.htm')
soup = BeautifulSoup(r.text, 'html.parser')
for entry in soup.findAll('tr', attrs={'class':'odd'}):
    print(entry.text.split('\n'))
    print(entry.find('a', href=True).get('href'))

['', '03/10/2018', 'Conservative', '', "Leader's speech, Birmingham 2018"]
speech-archive.htm?speech=370
['', '18/09/2018', 'Liberal Democrat', '', "Leader's speech, Brighton 2018 "]
speech-archive.htm?speech=368
['', '27/09/2017', 'Labour', '', "Leader's speech, Brighton 2017"]
speech-archive.htm?speech=366
['', '05/10/2016', 'Conservative', '', "Leader's speech, Birmingham 2016"]
speech-archive.htm?speech=364
['', '20/09/2016', 'Liberal Democrat', '', "Leader's speech, Brighton 2016"]
speech-archive.htm?speech=362
['', '29/09/2015', 'Labour', '', "Leader's speech, Brighton 2015"]
speech-archive.htm?speech=359
['', '08/10/2014', 'Liberal Democrat', 'Clegg, Nick', "Leader's speech, Glasgow 2014"]
speech-archive.htm?speech=357
['', '23/09/2014', 'Labour', 'Miliband, Ed', "Leader's speech, Manchester 2014"]
speech-archive.htm?speech=355
['', '24/09/2013', 'Labour', 'Miliband, Ed', "Leader's speech, Brighton 2013"]
speech-archive.htm?speech=353
['', '10/10/2012', 'Conservative', 'Cameron,

In [9]:
brit_pol = []
r = requests.get('http://www.britishpoliticalspeech.org/speech-archive.htm')
soup = BeautifulSoup(r.text, 'html.parser')
for entry in soup.findAll('tr', attrs={'class':'odd'}):
    info_list = entry.text.split('\n')
    brit_pol.append([info_list[1],
                   info_list[2],
                   info_list[3],
                   info_list[4],
                   'http://www.britishpoliticalspeech.org/' + entry.find('a', href=True).get('href')])

In [14]:
brit_pol_df = pd.DataFrame(brit_pol,columns=['date','party','speaker','title','link'])

In [15]:
brit_pol_df.head()

,date,party,speaker,title,link
0,03/10/2018,Conservative,,"Leader's speech, Birmingham 2018",http://www.britishpoliticalspeech.org/speech-a...
1,18/09/2018,Liberal Democrat,,"Leader's speech, Brighton 2018",http://www.britishpoliticalspeech.org/speech-a...
2,27/09/2017,Labour,,"Leader's speech, Brighton 2017",http://www.britishpoliticalspeech.org/speech-a...
3,05/10/2016,Conservative,,"Leader's speech, Birmingham 2016",http://www.britishpoliticalspeech.org/speech-a...
4,20/09/2016,Liberal Democrat,,"Leader's speech, Brighton 2016",http://www.britishpoliticalspeech.org/speech-a...


In [16]:
brit_pol_df.to_csv('brit_pol_df.csv',index=False)

In [2]:
brit_pol_df = pd.read_csv('brit_pol_df.csv')

In [16]:
r = requests.get('http://www.britishpoliticalspeech.org/speech-archive.htm?speech=370')
soup = BeautifulSoup(r.text, 'html.parser')
soup
# soup.find('p', attrs={'class':'speech-speaker'}).text
# soup.find('div', attrs={'class':'speech-content'}).text

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<link href="www.waters-creative.co.uk/favicon.ico" rel="shortcut icon"/> <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/> <title>British Political Speech | Speech Archive</title> <meta "rhetoric="" britain",="" british="" communication,="" content="british political speech, british politics, uk politics, speech archive, blair speech, cameron speech, thatcher speech, wilson speech, kinnock speech, heath speech, brown speech, speeches, political, British Political Speech, Swansea University, politics, rhetoric, " in="" name="keywords" oratory,="" political="" politics",="" rhetoric="" speech,="" speechwriting"=""/> <meta content="Welcome to the archive of british political speech. The archive contains speeches from 1895 to the present by political figures from all political parties.  " name="descr

In [7]:
brit_pol_df.tail(10)

,date,party,speaker,title,link
171,17/11/1909,Conservative,"Balfour, Arthur","Leader's speech, Manchester 1909",http://www.britishpoliticalspeech.org/speech-a...
172,19/06/1908,Liberal,"Asquith, Herbert","Leader's speech, Birmingham 1908",http://www.britishpoliticalspeech.org/speech-a...
173,14/11/1907,Conservative,"Balfour, Arthur","Leader's speech, Birmingham 1907",http://www.britishpoliticalspeech.org/speech-a...
174,27/07/1906,Conservative,"Balfour, Arthur","Leader's speech, London 1906",http://www.britishpoliticalspeech.org/speech-a...
175,19/05/1905,Liberal,"Campbell-Bannerman, Sir Henry","Leader's speech, Newcastle-on-Tyne 1905",http://www.britishpoliticalspeech.org/speech-a...
176,01/10/1903,Conservative,"Balfour, Arthur","Leader's speech, Sheffield 1903",http://www.britishpoliticalspeech.org/speech-a...
177,14/10/1902,Conservative,"Balfour, Arthur","Leader's speech, Manchester 1902",http://www.britishpoliticalspeech.org/speech-a...
178,18/12/1900,Conservative,"Salisbury, Lord","Leader's speech, London 1900",http://www.britishpoliticalspeech.org/speech-a...
179,16/11/1897,Conservative,"Salisbury, Lord","Leader's speech, London 1897",http://www.britishpoliticalspeech.org/speech-a...
180,27/03/1896,Liberal,Earl of Rosebery,"Leader's speech, Huddersfield 1896",http://www.britishpoliticalspeech.org/speech-a...


In [10]:
index_groups = range(brit_pol_df.shape[0],0,-19)
indexes = {group:[i for i in range(0,group) if group-i<20] for group in index_groups}

In [19]:
def get_text(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    text_speaker = soup.find('p', attrs={'class':'speech-speaker'}).text
    text = soup.find('div', attrs={'class':'speech-content'}).text
    return text_speaker, text

def get_speeches(index_group, indexes, return_dict):
    results = []
    for ind in tqdm_notebook(indexes):
        speaker = brit_pol_df.iloc[ind,2]
        party = brit_pol_df.iloc[ind,1]
        date = brit_pol_df.iloc[ind,0]
        title = brit_pol_df.iloc[ind,3]
        url = brit_pol_df.iloc[ind,4]
        text_speaker, text = get_text(url)
        results.append([speaker, party, date, title, text, text_speaker])
    return_dict[index_group]=results

def request_thread(index_groups):
    manager = mp.Manager()
    return_dict = manager.dict()
    threads = []
    for index_group, indexes in tqdm_notebook(index_groups.items()):
        thread = threading.Thread(name=index_group, 
                                  target=get_speeches, 
                                  args=(index_group, indexes, return_dict))
        thread.start()
        threads.append(thread)

    for t in threads:
        t.join()

    return return_dict

In [20]:
brit_pol_dict = request_thread(indexes)

In [21]:
brit_pol_df = pd.DataFrame([x for group in brit_pol_dict.values() for x in group],columns=['speaker', 'party', 'date', 'title', 'text', 'text_speaker'])

In [22]:
brit_pol_df

,speaker,party,date,title,text,text_speaker
0,NaN,Conservative,03/10/2018,"Leader's speech, Birmingham 2018",Thank you very much for that warm welcome. You...,Theresa May (Conservative)
1,NaN,Liberal Democrat,18/09/2018,"Leader's speech, Brighton 2018","\n \nConference, we meet at an absolutely cru...",Vince Cable (Liberal Democrat)
2,NaN,Labour,27/09/2017,"Leader's speech, Brighton 2017","We meet here this week as a united Party, adva...",Jeremy Corbyn (Labour)
3,NaN,Conservative,05/10/2016,"Leader's speech, Birmingham 2016",The new centre ground\n \nWhen we came to Birm...,Theresa May (Conservative)
4,NaN,Liberal Democrat,20/09/2016,"Leader's speech, Brighton 2016",Preston\n \nLiberal Democrats are good at lots...,Tim Farron (Liberal Democrat)
5,NaN,Labour,29/09/2015,"Leader's speech, Brighton 2015","Friends, thank you so much for that incredible...",Jeremy Corbyn (Labour)
6,"Clegg, Nick",Liberal Democrat,08/10/2014,"Leader's speech, Glasgow 2014","Before I say anything else, I’m sure I speak o...",Nick Clegg (Liberal Democrat)
7,"Miliband, Ed",Labour,23/09/2014,"Leader's speech, Manchester 2014",It’s great to be at Labour Party Conference in...,Ed Miliband (Labour)
8,"Miliband, Ed",Labour,24/09/2013,"Leader's speech, Brighton 2013",It's great to be in Brighton. And I want to st...,Ed Miliband (Labour)
9,"Cameron, David",Conservative,10/10/2012,"Leader's speech, Birmingham 2012","In May 2010, this party stood on the threshold...",David Cameron (Conservative)


In [23]:
brit_pol_df.to_csv('brit_pol_df.csv', index=False)